# How to optimize the Meta Llama-3 70B Amazon JumpStart model for inference using Amazon SageMaker model optimization jobs
**Recommended kernel(s):** This notebook can be run with any Amazon SageMaker Studio kernel.

In this notebook, you will learn how to apply state-of-the-art optimization techniques to an Amazon JumpStart model (JumpStart model ID: `meta-textgeneration-llama-3-70b`) using Amazon SageMaker ahead-of-time (AOT) model optimization capabilities. Each example includes the deployment of the optimized model to an Amazon SageMaker endpoint. In all cases, the inference image will be the SageMaker-managed [LMI (Large Model Inference)](https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-container-docs.html) Docker image. LMI images features a [DJL serving](https://github.com/deepjavalibrary/djl-serving) stack powered by the [Deep Java Library](https://djl.ai/). 

You will successively:
1. Deploy a pre-optimized variant of the Amazon JumpStart model with speculative decoding enabled (using SageMaker provided draft model). For popular models, the JumpStart team indeed selects and applies the best optimization configurations for you.


**Notices:**
* Make sure that the `ml.p4d.24xlarge` and `ml.inf2.48xlarge` instance types required for the tutorials are available in your AWS Region.
* Make sure that the value of your "ml.p4d.24xlarge for endpoint usage" and "ml.inf2.48xlarge for endpoint usage" Amazon SageMaker service quotas allow you to deploy at least one Amazon SageMaker endpoint using these instance types.

This notebook leverages the [Model Builder Class](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-modelbuilder-creation.html) within the [`sagemaker` Python SDK](https://sagemaker.readthedocs.io/en/stable/index.html) to abstract out container and model server management/tuning. Via the Model Builder Class you can easily interact with JumpStart Models, HuggingFace Hub Models, and also custom models via pointing towards an S3 path with your Model Data. For this sample we will focus on the JumpStart Optimization path.

### License agreement
* This model is under the Meta license, please refer to the original model card.
* This notebook is a sample notebook and not intended for production use.

### Execution environment setup
This notebook requires the following third-party Python dependencies:
* AWS [`boto3`](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html#)
* AWS [`sagemaker`](https://sagemaker.readthedocs.io/en/stable/index.html) with a version greater than or equal to 2.225.0 

Let's install or upgrade these dependencies using the following command:

In [12]:
%pip install sagemaker>=2.225.0 boto3 huggingface_hub --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.24 requires botocore==1.34.142, but you have botocore 1.34.150 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


### Setup

In [13]:
import boto3
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.session import Session
import logging
import huggingface_hub
from pathlib import Path

In [14]:
sagemaker_session = Session()

artifacts_bucket_name = sagemaker_session.default_bucket()
execution_role_arn = sagemaker_session.get_caller_identity_arn()

js_model_id = "meta-textgeneration-llama-3-70b"
gpu_instance_type = "ml.p4d.24xlarge"
neuron_instance_type = "ml.inf2.48xlarge"

In [15]:
response = "Hello, I'm a language model, and I'm here to help you with your English."

sample_input = {
    "inputs": "Hello, I'm a language model,",
    "parameters": {"max_new_tokens": 128, "top_p": 0.9, "temperature": 0.6},
}

sample_output = [{"generated_text": response}]

schema_builder = SchemaBuilder(sample_input, sample_output)

## 1. Deploy a pre-optimized deployment configuration with speculative decoding (SageMaker provided draft model)
The `meta-textgeneration-llama-3-70b` JumpStart model is available with multiple pre-optimized deployment configuration. Optimized model artifacts for each configuration have already been created by the JumpStart team and a readily available for deployment. In this section, you will deploy one of theses pre-optimized configuration to an Amazon SageMaker endpoint. 

### What is speculative decoding?
Speculative decoding is an inference optimization technique introduced by [Y. Leviathan et al. (ICML 2023)](https://arxiv.org/abs/2211.17192) used to accelerate the decoding process of large and therefore slow LLMs for latency-critical applications. The key idea is to use a smaller, less powerful but faster model called the ***draft model*** to generate candidate tokens that get validated by the larger, more powerful but slower model called the ***target model***. At each iteration, the draft model generates $K>1$ candidate tokens. Then, using a single forward pass of the larger target model, none, part, or all candidate tokens get accepted. The more aligned the selected draft model is with the target model, the better guesses it makes, the higher candidate token acceptance rate and therefore the higher the speed ups. The larger the size gap between the target and the draft model, the largest the potential speedups.

Let's start by creating a `ModelBuilder` instance for the model:

In [16]:
model_builder = ModelBuilder(
    model=js_model_id,
    schema_builder=schema_builder,
    sagemaker_session=sagemaker_session,
    role_arn=execution_role_arn,
    log_level=logging.ERROR
)

For each optimization configuration, the JumpStart team has computed key performance metrics such as time-to-first-token (TTFT) latency and throughput for multiple hardwares and concurrent invocation intensities. Let's visualize these metrics using the `display_benchmark_metrics` method:

In [17]:
model_builder.display_benchmark_metrics()

Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
INFO:sagemaker.jumpstart:Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
Using model 'meta-textgeneration-llama-3-70b' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
No instance type selected for inference hosting endpoint. Defaulting to ml.p4d.24xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.p4d.24xlarge.
ModelBuilder: INFO:     JumpStart ID meta-textgeneration-llama-3-70b is packaged with Image URI: 763104351884.dkr

| Instance Type             | Config Name   |   Concurrent Users |   Latency, TTFT (P50 in sec) |   Throughput (P50 in tokens/sec/user) |
|:--------------------------|:--------------|-------------------:|-----------------------------:|--------------------------------------:|
| ml.g5.48xlarge            | lmi           |                  1 |                         2.02 |                                 18.80 |
| ml.g5.48xlarge            | lmi           |                  2 |                         2.10 |                                 15.40 |
| ml.g5.48xlarge            | lmi           |                  4 |                         2.15 |                                  9.40 |
| ml.g5.48xlarge            | lmi           |                  8 |                         2.93 |                                  6.70 |
| ml.p4d.24xlarge           | lmi           |                 64 |                         0.20 |                                  9.70 |
| ml.p4d.24xlarge           | lmi 

Now, let's pick and deploy the `lmi-optimized` pre-optimized configuration to a `ml.p4d.24xlarge` instance. The `lmi-optimized` configuration enables speculative decoding. In this configuration, a SageMaker provided draft model is used. Therefore, you don't have to supply a draft model. 

In [18]:
model_builder.set_deployment_config(config_name="lmi-optimized", instance_type="ml.g5.48xlarge")

Currently set deployment configuration can be visualized using the `get_deployment_config` method:

In [21]:
model_builder.get_deployment_config()

Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
INFO:sagemaker.jumpstart:Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
Using model 'meta-textgeneration-llama-3-70b' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
Instance rate metrics will be omitted. Reason: User: arn:aws:sts::969468890356:assumed-role/mlopt3/SageMaker is not authorized to perform: pricing:GetProducts because no identity-based policy allows the pricing:GetProducts action


{'DeploymentConfigName': 'lmi-optimized',
 'DeploymentArgs': {'ImageUri': '763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.28.0-lmi10.0.0-cu124',
  'ModelData': {'S3DataSource': {'S3Uri': 's3://jumpstart-private-cache-prod-us-east-1/meta-textgeneration/meta-textgeneration-llama-3-70b/artifacts/inference-prepack/v1.1.0/',
    'S3DataType': 'S3Prefix',
    'CompressionType': 'None'}},
  'ModelPackageArn': None,
  'Environment': {'SAGEMAKER_PROGRAM': 'inference.py',
   'ENDPOINT_SERVER_TIMEOUT': '3600',
   'MODEL_CACHE_ROOT': '/opt/ml/model',
   'SAGEMAKER_ENV': '1',
   'HF_MODEL_ID': '/opt/ml/model',
   'OPTION_SPECULATIVE_DRAFT_MODEL': '/opt/ml/additional-model-data-sources/draft_model',
   'SAGEMAKER_MODEL_SERVER_WORKERS': '1'},
  'InstanceType': 'ml.g5.48xlarge',
  'ComputeResourceRequirements': {'MinMemoryRequiredInMb': 393216,
   'NumberOfAcceleratorDevicesRequired': 8},
  'ModelDataDownloadTimeout': 1200,
  'ContainerStartupHealthCheckTimeout': 1200,
  'AdditionalDataS

Now, let's build the `Model` instance and use it to deploy the selected optimized configuration. This operation may take a few minutes.

In [20]:
optimized_model = model_builder.build()

ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
INFO:sagemaker.jumpstart:Model 'meta-textgeneration-llama-3-70b' requires accepting end-user license agreement (EULA). See https://jumpstart-cache-prod-us-east-1.s3.us-east-1.amazonaws.com/fmhMetadata/eula/llama3Eula.txt for terms of use.
Using model 'meta-textgeneration-llama-3-70b' with wildcard version identifier '*'. You can pin to version '2.2.1' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
ModelBuilder: INFO:     JumpStart Model ID detected.
ModelBuilder: INFO:     JumpStart Model ID detected.


In [ ]:
predictor = optimized_model.deploy(accept_eula=True)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
INFO:sagemaker:Creating model with name: meta-textgeneration-llama-3-70b-2024-07-29-20-58-58-076
INFO:sagemaker:Creating endpoint-config with name meta-textgeneration-llama-3-70b-2024-07-30-01-03-20-173
INFO:sagemaker:Creating endpoint with name meta-textgeneration-llama-3-70b-2024-07-30-01-03-20-173


-----------

Once the deployment has finished successfully, you can send queries to the model by simply using the predictor's `predict` method:

In [ ]:
predictor.predict(sample_input)

In [ ]:
# Clean up
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)